# Examples and Validation Tests

This notebook is used to generate [vr-spec](https://vr-spec.readthedocs.io/) examples and the tests in [vr-spec/validation/](https://github.com/ga4gh/vr-spec/tree/master/validation).

To Do:
* use real examples from ApoE for Alleles, Haplotypes, and Genotypes; e.g.. 4 alleles, 2 Haplotypes, 1 Genotype
* test order invariance
* test inline v. reference objects


In [10]:
from ga4gh.core import ga4gh_digest, ga4gh_identify, ga4gh_serialize, sha512t24u
from ga4gh.vrs import __version__, models, normalize
__version__

'0.6.0rc1.dev7+g1a5bd28'

In [11]:
import json
import yaml
def filter_dict(d):
    try:
        return {k: filter_dict(d[k])
                for k in d
                if not k.startswith("_")}
    except:
        return d
def as_str(s):
    return s if isinstance(s, str) else s.decode()
def dj(o):
    """print VR object as pretty formated json"""
    print(json.dumps(filter_dict(o.as_dict()), indent=2, sort_keys=True))
def dy(fns, o):
    """execute function f in fns on o, returning a yaml block representing the test"""
    r = {
        "in": o.as_dict(),
        "out": {f.__name__: as_str(f(o)) for f in fns}
    }
    print(yaml.dump(filter_dict({o.type._value: {"-": r}})).replace("'-':","-"))

----
# Generate sample objects

In [12]:
# SimpleInterval
simple_interval = models.SimpleInterval(start=44908821, end=44908822)
dj(simple_interval)

{
  "end": 44908822,
  "start": 44908821,
  "type": "SimpleInterval"
}


In [13]:
# SequenceLocation based on a SimpleInterval
sequence_location = models.SequenceLocation(
    sequence_id="ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
    interval=simple_interval)
dj(sequence_location)

{
  "interval": {
    "end": 44908822,
    "start": 44908821,
    "type": "SimpleInterval"
  },
  "sequence_id": "ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
  "type": "SequenceLocation"
}


In [14]:
# SequenceState
sequence_state = models.SequenceState(sequence="T")
dj(sequence_state)

{
  "sequence": "T",
  "type": "SequenceState"
}


In [15]:
# Text
text_variation = models.Text(definition="APOE loss")
dj(text_variation)

{
  "definition": "APOE loss",
  "type": "Text"
}


In [16]:
# Allele
allele = models.Allele(location=sequence_location,
                       state=models.SequenceState(sequence="T"))
dj(allele)

{
  "location": {
    "interval": {
      "end": 44908822,
      "start": 44908821,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
    "type": "SequenceLocation"
  },
  "state": {
    "sequence": "T",
    "type": "SequenceState"
  },
  "type": "Allele"
}


In [17]:
# Haplotype
haplotype = models.Haplotype(members=[allele])
dj(haplotype)

{
  "members": [
    {
      "location": {
        "interval": {
          "end": 44908822,
          "start": 44908821,
          "type": "SimpleInterval"
        },
        "sequence_id": "ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
        "type": "SequenceLocation"
      },
      "state": {
        "sequence": "T",
        "type": "SequenceState"
      },
      "type": "Allele"
    }
  ],
  "type": "Haplotype"
}


----
# Functions

### Truncated Digest (sha512t24u)

In [19]:
sha512t24u(b"")

'z4PhNX7vuL3xVChQ1m2AB9Yg5AULVxXc'

In [20]:
sha512t24u(b"ACGT")

'aKF498dAxcJAqme6QYQ7EZ07-fiw8Kw2'

### Digest Serialization (`ga4gh_serialize`)

The ga4gh digest serialization form is like json, but it the specification ensures that all implementations will produce the same binary payload.

In [21]:
allele = models.Allele(location=models.SequenceLocation(
    sequence_id="ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
    interval=simple_interval),
    state=models.SequenceState(sequence="T"))
ga4gh_serialize(allele)

b'{"location":"u5fspwVbQ79QkX6GHLF8tXPCAXFJqRPx","state":{"sequence":"T","type":"SequenceState"},"type":"Allele"}'

In [22]:
dj(allele)

{
  "location": {
    "interval": {
      "end": 44908822,
      "start": 44908821,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl",
    "type": "SequenceLocation"
  },
  "state": {
    "sequence": "T",
    "type": "SequenceState"
  },
  "type": "Allele"
}


### Object Digest (`ga4gh_digest`)
VR computed identifiers are constructed from digests on serialized objects by prefixing a VR digest with a type-specific code.

In [23]:
# applying ga4gh_digest to the serialized allele returns a base64url-encoded digest
ga4gh_digest(allele)

'EgHPXXhULTwoP4-ACfs-YCXaeUQJBjH_'

In [24]:
# Which is equivalent to
sha512t24u(ga4gh_serialize(allele))

'EgHPXXhULTwoP4-ACfs-YCXaeUQJBjH_'

### Object Computed Identifier (`ga4gh_identify`)

In [25]:
ga4gh_identify(allele)

'ga4gh:VA.EgHPXXhULTwoP4-ACfs-YCXaeUQJBjH_'

----
## External Data

In [26]:
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
seqrepo_rest_service_url = "http://localhost:5000/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

In [27]:
def get_sequence(identifier, start=None, end=None):
    """returns sequence for given identifier, optionally limited to interbase <start, end> interval"""
    return dp.get_sequence(identifier, start, end)
def get_sequence_length(identifier):
    """return length of given sequence identifier"""
    return dp.get_metadata(identifier)["length"]
def translate_sequence_identifier(identifier, namespace):
    """return for given identifier, return *list* of equivalent identifiers in given namespace"""
    return dp.translate_sequence_identifier(identifier, namespace)

In [28]:
get_sequence_length("ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl")

58617616

In [29]:
start, end = 44908821-25, 44908822+25
get_sequence("ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl", start, end)

'CCGCGATGCCGATGACCTGCAGAAGCGCCTGGCAGTGTACCAGGCCGGGGC'

In [30]:
translate_sequence_identifier("GRCh38:19", "ga4gh")

['ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl']

In [31]:
translate_sequence_identifier("ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl", "GRCh38")

['GRCh38:chr19', 'GRCh38:19']

----
# Validation examples

## models.yaml

In [33]:
for o in [simple_interval]:
    dy([ga4gh_serialize], o)
for o in [sequence_location, allele, haplotype]:
    dy([ga4gh_serialize, ga4gh_digest, ga4gh_identify], o)

SimpleInterval:
  -
    in:
      end: 44908822
      start: 44908821
      type: SimpleInterval
    out:
      ga4gh_serialize: '{"end":44908822,"start":44908821,"type":"SimpleInterval"}'

SequenceLocation:
  -
    in:
      interval:
        end: 44908822
        start: 44908821
        type: SimpleInterval
      sequence_id: ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl
      type: SequenceLocation
    out:
      ga4gh_digest: u5fspwVbQ79QkX6GHLF8tXPCAXFJqRPx
      ga4gh_identify: ga4gh:VSL.u5fspwVbQ79QkX6GHLF8tXPCAXFJqRPx
      ga4gh_serialize: '{"interval":{"end":44908822,"start":44908821,"type":"SimpleInterval"},"sequence_id":"IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl","type":"SequenceLocation"}'

Allele:
  -
    in:
      location:
        interval:
          end: 44908822
          start: 44908821
          type: SimpleInterval
        sequence_id: ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl
        type: SequenceLocation
      state:
        sequence: T
        type: SequenceState
      type